In [1]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

# data gen

In [2]:
import random as rand
import numpy as np


def data_gen(num_rbf):
    data = []
    clt_list = []
    std_list = []
    for i in range(num_rbf):
        clt = rand.random()
        std = rand.random()
        clt_list.append(clt)
        std_list.append(std)
        for i in [-5, -4, -3, -2, -1, 0 ,1, 2, 3, 4]:
            data_i = np.exp(-(i - clt) * (i - clt) / 2 * (std * std))
            data.append(data_i)
    
    return torch.tensor(data, device = device).float(), torch.tensor(clt_list, device = device).float(), torch.tensor(std_list, device = device).float()

In [3]:
data, clt_list, std_list = data_gen(10)

In [4]:
data

tensor([1.8216e-02, 6.8772e-02, 1.9884e-01, 4.4031e-01, 7.4669e-01, 9.6976e-01,
        9.6455e-01, 7.3473e-01, 4.2862e-01, 1.9150e-01, 9.9587e-01, 9.9718e-01,
        9.9824e-01, 9.9905e-01, 9.9962e-01, 9.9993e-01, 9.9999e-01, 9.9980e-01,
        9.9937e-01, 9.9868e-01, 3.8496e-05, 9.7527e-04, 1.3337e-02, 9.8453e-02,
        3.9230e-01, 8.4382e-01, 9.7972e-01, 6.1402e-01, 2.0773e-01, 3.7935e-02,
        7.9060e-01, 8.5153e-01, 9.0436e-01, 9.4706e-01, 9.7793e-01, 9.9571e-01,
        9.9967e-01, 9.8963e-01, 9.6601e-01, 9.2980e-01, 1.2255e-05, 4.0046e-04,
        6.8949e-03, 6.2547e-02, 2.9895e-01, 7.5284e-01, 9.9890e-01, 6.9832e-01,
        2.5722e-01, 4.9919e-02, 2.9740e-05, 8.7708e-04, 1.3341e-02, 1.0466e-01,
        4.2347e-01, 8.8372e-01, 9.5115e-01, 5.2800e-01, 1.5117e-01, 2.2322e-02,
        2.3320e-05, 1.0731e-03, 2.1152e-02, 1.7856e-01, 6.4564e-01, 9.9989e-01,
        6.6325e-01, 1.8843e-01, 2.2930e-02, 1.1951e-03, 9.3332e-01, 9.5679e-01,
        9.7545e-01, 9.8901e-01, 9.9724e-

In [5]:
clt_list

tensor([0.4798, 0.7487, 0.7422, 0.7819, 0.9413, 0.6111, 0.0159, 0.0019, 0.4662,
        0.2335], device='cuda:0')

In [6]:
std_list

tensor([0.5165, 0.0158, 0.7852, 0.1186, 0.8005, 0.8137, 0.9208, 0.0743, 0.8768,
        0.4732], device='cuda:0')

# clt, std 고정

In [7]:
class RBF(nn.Module):
    def __init__(self, clt, std):
        super(RBF, self).__init__()
        
        self.rbf_clt = clt
        self.rbf_std = std
        
    
    def rbf(self, x, cluster, std):
        return torch.exp(-(x - cluster) * (x - cluster) / 2 * (std * std))
    
    
    def forward(self, x):     
        
        x = self.rbf(x, self.rbf_clt, self.rbf_std)
    
        return x

In [8]:
RBF(clt_list[0], std_list[0])(data[0]) * torch.rand(1, device = device)

tensor([0.7811], device='cuda:0')

In [25]:
a = sum([RBF(clt_list[0], std_list[0])(data[0]) * torch.rand(1, device = device), RBF(clt_list[1], std_list[0])(data[1]) * torch.rand(1, device = device)])

In [24]:
a

tensor([0.3426, 0.3426], device='cuda:0')

In [30]:
a.reshape()

RuntimeError: shape '[2]' is invalid for input of size 1

In [11]:
a = torch.cat([a,a])

In [12]:
a

tensor([0.3426, 0.3426], device='cuda:0')

In [31]:
class RBFnetwork(nn.Module):
    def __init__(self, timelag1, time2, clt, std):
        super(RBFnetwork, self).__init__()
        torch.cuda.manual_seed(0)
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.timelag1 = timelag1 # num_rbf
        self.init_weight = nn.Parameter(torch.rand(self.timelag1))
        self.timelag2 = time2 # number of data in one rbf 
        
        self.clt_list = clt
        self.std_list = std
        self.rbf_list = [RBF(self.clt_list[i], self.std_list[i]) for i in range(self.timelag1)]
        
    def forward(self, x):

        for j in range(self.timelag1 * self.timelag2):
            if j ==0:
                y = sum([self.init_weight[i] * self.rbf_list[i](x[j]) for i in range(self.timelag1)])
                y.reshape(1)
            else:
                y = torch.cat([y, sum([self.init_weight[i] * self.rbf_list[i](x[j]) for i in range(self.timelag1)])])
        
        return y

In [15]:
def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params
        
def train_RBFlayer(model, input_, target, lr, epochs, lookback = 5, device = device):
    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    
    train_loss_list = []
    
    best_it = None
    best_model = None
    best_loss = np.inf
    
    target_list = []
    
    for j in range(len(target) - 2):
        target_list.append((target[j+2] - target[j])/2)
    
    loss_list = []
    cause_list = []
    for epoch in range(epochs):
        cause = model(input_)
        cause_list.append(cause)
        grad = []
        
        
        for i in range(len(cause) - 2):
            grad.append((cause[i+2] - cause[i])/2)
        
        loss1 = sum([loss_fn(grad[i], target_list[i]) for i in range(len(grad))])
        loss2 = sum([loss_fn(cause[i], target[i]) for i in range(len(input_))])
        
        loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()
        model.zero_grad()
        
        loss_list.append(loss)
        mean_loss = loss / len(grad)
        train_loss_list.append(mean_loss)
        
        if mean_loss < best_loss:
            best_loss = mean_loss
            best_it = epoch
            best_model = deepcopy(model)
            
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break
        print("epoch {} cause loss {} :".format(epoch, loss / len(input_)))
        print('gradient loss :', loss1/len(grad))
        print('value loss :', loss2/len(input_))
                
    best_cause = cause_list[best_it]    
    restore_parameters(model, best_model)

    return best_model, loss_list, best_cause

In [32]:
import time

cause_list = []

start = time.time()
model = RBFnetwork(timelag1 = 10, time2 = 10, clt = clt_list, std = std_list)
best_model, loss_list, best_cause = train_RBFlayer(model, data, data, 0.01, 1000, device)
cause_list.append(best_cause.cpu().detach().numpy())
print("time :", time.time() - start)
print('-------------------------------------------------------------------------------------------')

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated